In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
from PIL import Image
from torchvision.transforms import v2
from torchvision import tv_tensors

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# DATASET CLASS
class ParkingDataset(Dataset):
     #init func constructor
    def __init__(self, root, annFile, transforms=None):

        self.root = root #director path
        self.coco = COCO(annFile) #loading coco format file
        self.transforms = transforms
        all_ids = list(sorted(self.coco.imgs.keys())) #taking all image id s from coco annotations file
        self.ids = [] #id storing for only valid images

        print("file existing checking.....")
        for img_id in all_ids:

            img_info = self.coco.loadImgs(img_id)[0] # take image info
            path = img_info['file_name'] #image name
            full_path = os.path.join(self.root, path) #full name of image path

            # if image file exists and valid
            if os.path.exists(full_path):
                self.ids.append(img_id)
            else:
                # error if image is not valid
                print(f"Warning : {path} not valid...")

        print(f"check is completed. Total :{len(all_ids)} images , and  {len(self.ids)} : are valid.")

    # taking a sinle example
    def __getitem__(self, index):

        img_id = self.ids[index] #taking image with that index
        ann_ids = self.coco.getAnnIds(imgIds=img_id) # take all annotation ids for this image
        coco_annotation = self.coco.loadAnns(ann_ids) # get annotation details

        # gte image info
        img_info = self.coco.loadImgs(img_id)[0]
        path = img_info['file_name']
        img = Image.open(os.path.join(self.root, path)).convert("RGB") # open image and convert to RGB

        boxes = []
        masks = []
        labels = []

        # iterate over annotations
        for ann in coco_annotation:
            # COCO bbox format: [xmin, ymin, width, height]
            xmin, ymin, w, h = ann['bbox']
            boxes.append([xmin, ymin, xmin + w, ymin + h]) #appropiate format for mask r cnn
            labels.append(ann['category_id']) # 1: empty, 2: occupied
            masks.append(self.coco.annToMask(ann)) #turn it to  mask for mask r cnn

        # if image is empty creating empty values zeros
        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.as_tensor([], dtype=torch.int64)
            masks = torch.zeros((0, img.size[1], img.size[0]), dtype=torch.uint8)
        else: #convert tensor
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            masks = torch.as_tensor(np.array(masks), dtype=torch.uint8)

        # target for  tv_tensorsv2
        target = {
            "boxes": tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=img.size[::-1]),
            "masks": tv_tensors.Mask(masks),
            "labels": labels,
            "image_id": torch.tensor([img_id])
        }

        if self.transforms:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.ids)

# Transfroms and resızes
def get_transform(train=True):
    transforms = []

    # (H=500, W=1100)
    transforms.append(v2.Resize((500, 1100)))
    transforms.append(v2.ToImage())

    if train:
        # in train mode augmentation
        transforms.append(v2.RandomHorizontalFlip(p=0.5))
        transforms.append(v2.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1))
        transforms.append(v2.RandomPerspective(distortion_scale=0.2, p=0.4))

        transforms.append(v2.RandomApply([v2.GaussianBlur(kernel_size=(5, 5))], p=0.05))
        transforms.append(v2.SanitizeBoundingBoxes()) # removing invalid bounding boxes after transform

    transforms.append(v2.ToDtype(torch.float32, scale=True))
    #composing all transforms
    return v2.Compose(transforms)

# visualize func for checking data : boxes rezize transform match
def visualize_check(dataset, index=0):
    img, target = dataset[index] #take a data

    # Tensor to Numpy
    img_np = img.permute(1, 2, 0).numpy()



    if len(target["masks"]) > 0:
        mask_all = target["masks"].sum(0).numpy()
        mask_all = np.where(mask_all > 0, 1, 0)
    else:
        mask_all = np.zeros((800, 1000))

    plt.figure(figsize=(12, 8))
    plt.imshow(img_np)
    plt.imshow(mask_all, alpha=0.3, cmap='jet')
    plt.title(f"Check - Labels: {target['labels'].tolist()}")
    plt.axis('off')
    plt.show()



# directory path
root_path = '/content/drive/MyDrive/train/'
ann_path = '/content/drive/MyDrive/train/_annotations.coco.json'

# Dataset
train_dataset = ParkingDataset(root_path, ann_path, transforms=get_transform(train=True))
val_dataset = ParkingDataset(root_path, ann_path, transforms=get_transform(train=False)) #without augmentation



print(f"Total image count: {len(train_dataset)}")

# visualize_check(train_dataset, index=0)

loading annotations into memory...
Done (t=11.59s)
creating index...
index created!
file existing checking.....
check is completed. Total :3116 images , and  3115 : are valid.
loading annotations into memory...
Done (t=0.13s)
creating index...
index created!
file existing checking.....
check is completed. Total :3116 images , and  3115 : are valid.
Total image count: 3115


In [4]:
from torch.utils.data import Subset
import numpy as np

# Total data
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(0.2 * dataset_size)) # %20 validation

np.random.seed(42) # same splitting
np.random.shuffle(indices)

val_indices = indices[:split]
train_indices = indices[split:]

# splitting with subset
train_sub_dataset = Subset(train_dataset, train_indices)
val_sub_dataset = Subset(val_dataset, val_indices)

# Loaders
train_loader = DataLoader(
    train_sub_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x))
)

val_loader = DataLoader(
    val_sub_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x))
)

print(f"Train sample size: {len(train_sub_dataset)}")
print(f"Val sample size: {len(val_sub_dataset)}")

Train sample size: 2492
Val sample size: 623


In [5]:
def set_training_phase(model, phase_id):
    """freezing method with 4 phase"""
    # freeze everything
    for param in model.parameters():
        param.requires_grad = False

    if phase_id == 1:
        # phase 1: train only roı heads (classification, bbox, mask heads)
        for param in model.roi_heads.parameters(): param.requires_grad = True
        for param in model.rpn.parameters(): param.requires_grad = True
        print(">>> FAZ 1: just heads.")

    elif phase_id == 2:
        # phase 2: ResNet Stage 4 and heads (layer4 + heads)
        for param in model.roi_heads.parameters(): param.requires_grad = True
        for param in model.rpn.parameters(): param.requires_grad = True
        for param in model.backbone.body.layer4.parameters(): param.requires_grad = True
        print(">>> FAZ 2: (layer4 + heads)")

    elif phase_id == 3:
        # phase 3: ResNet Stage 3 and upper layers (layer3 + layer4 + heads)
        for param in model.roi_heads.parameters(): param.requires_grad = True
        for param in model.rpn.parameters(): param.requires_grad = True
        for param in model.backbone.body.layer4.parameters(): param.requires_grad = True
        for param in model.backbone.body.layer3.parameters(): param.requires_grad = True
        print(">>> phase 3: (layer3 + layer4 + heads).")

    elif phase_id == 4:
        # phase 4: Full Fine-tuning
        for param in model.parameters(): param.requires_grad = True
        print(">>> phase 4: Tüm model eğitiliyor.")

def get_phase_config(epoch):
    if epoch < 10:    return 1, 0.001   # phase 1: Heads
    if epoch < 18:   return 2, 0.0008   # phase 2: Stage 4+
    if epoch < 30:   return 3, 0.0001  # phase 3: Stage 3+
    return 4, 0.00001                  # phase 4: Full (small LR)

In [6]:
import torchvision
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator , RPNHead
def get_model_instance_segmentation(num_classes , freeze_backbone=True):
    # load a pretrained Mask R-CNN model with ResNet50-FPN backbone
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(
        weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT,
        min_size=500,
        max_size=1100
    )



    # anchor size settins
    # 16 ve 32: for small boxes
    # 128, 256, 512, 1024: for large booxes
    anchor_sizes = ((16,), (32,), (64,), (128,), (256,))

    # aspect ratio settings
    # To cover horizontal, square, and vertical parking slots
    aspect_ratios = ((0.2, 1.0, 5.0),) * 5
    anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios) #create generator
    model.rpn.anchor_generator = anchor_generator #replace generator


    # RPN proposal tuning
    # ı had a problem about lot of samll boxes increased the number of candidates for large boxes
    model.rpn._pre_nms_top_n['training'] = 3000
    model.rpn._pre_nms_top_n['testing'] = 2000
    model.rpn._post_nms_top_n['training'] = 2000
    model.rpn._post_nms_top_n['testing'] = 1000


    # Box classification & regression head
    in_features = model.roi_heads.box_predictor.cls_score.in_features #same input feature
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) # replace with correct number of classes

    # Mask prediction head
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)



    return model

In [7]:
import torch
from tqdm.auto import tqdm
import time
import matplotlib.pyplot as plt
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_classes = 3
model = get_model_instance_segmentation(num_classes).to(device)

num_epochs = 40
all_train_losses = []
all_val_losses = []
best_val_loss = float('inf')
patience = 12
trigger_times = 0
current_phase = 0

#main trainig loop
for epoch in range(num_epochs):

    # determine training phase and LR
    phase_id, lr = get_phase_config(epoch)

    # if phase has changed
    if phase_id != current_phase:
        # Set which layers are trainable
        set_training_phase(model, phase_id)
        # give only trainable parameters to optimizer
        optimizer = AdamW(
            [p for p in model.parameters() if p.requires_grad],
            lr=lr,
            weight_decay=0.1
        )
        current_phase = phase_id
        trigger_times = 0
        print(f"--- Optimizer updated: Phase {phase_id}, LR {lr} ---")

    # train time
    model.train() #train mode
    epoch_train_loss = 0
    pbar_train = tqdm(train_loader, desc=f"Epoch {epoch+1} [Phase {phase_id}]") #progress bar

    for images, targets in pbar_train:

        images = list(img.to(device) for img in images) # move images to device
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets] # move target tensors to device

        loss_dict = model(images, targets)

        # Loss settings
        losses = (
            loss_dict['loss_classifier'] * 1.0 +
            loss_dict['loss_box_reg'] * 2.5 +
            loss_dict['loss_mask'] * 1.2 +
            loss_dict['loss_objectness'] * 1.0 +
            loss_dict['loss_rpn_box_reg'] * 1.0
        )

        optimizer.zero_grad()
        # Backpropagation

        losses.backward()
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
        optimizer.step()

        epoch_train_loss += losses.item()
        pbar_train.set_postfix({"loss": f"{losses.item():.4f}", "lr": lr})

    avg_train_loss = epoch_train_loss / len(train_loader)
    all_train_losses.append(avg_train_loss)

    # VALIDATION
    epoch_val_loss = 0
    # without gradient computation
    with torch.no_grad():
        # Mask R-CNN requires train() mode even for validation loss
        pbar_val = tqdm(val_loader, desc=f"Epoch {epoch+1} [VAL]")
        for images, targets in pbar_val:
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            val_loss_dict = model(images, targets)
            val_losses = sum(loss for loss in val_loss_dict.values())
            epoch_val_loss += val_losses.item()
            pbar_val.set_postfix({"val_loss": f"{val_losses.item():.4f}"})

    avg_val_loss = epoch_val_loss / len(val_loader)
    all_val_losses.append(avg_val_loss)

    print(f"\n>> Epoch {epoch+1} Summary: Phase: {phase_id} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    #  EARLY STOPPING AND MODEL SAVİNG
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        trigger_times = 0
        folder_path = "/content/drive/MyDrive/ParkModeli"
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        save_path = os.path.join(folder_path, "best_model_val.pth")
        torch.save(model.state_dict(), save_path)
        print(f"*** Val Loss improved model saved ***")
    else:
        trigger_times += 1
        print(f"Val Loss did not improved. Patience: {trigger_times}/{patience}")
        if trigger_times >= patience:
            print("Early Stopping !")
            break

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


100%|██████████| 170M/170M [00:01<00:00, 117MB/s]


>>> FAZ 1: just heads.
--- Optimizer updated: Phase 1, LR 0.001 ---


Epoch 1 [Phase 1]:   0%|          | 0/312 [00:00<?, ?it/s]

Epoch 1 [VAL]:   0%|          | 0/78 [00:00<?, ?it/s]

...

Epoch 29 [VAL]:   0%|          | 0/78 [00:00<?, ?it/s]


>> Epoch 29 Summary: Phase: 3 | Train Loss: 0.4943 | Val Loss: 0.3967
Val Loss did not improved. Patience: 11/12


Epoch 30 [Phase 3]:   0%|          | 0/312 [00:00<?, ?it/s]

Epoch 30 [VAL]:   0%|          | 0/78 [00:00<?, ?it/s]


>> Epoch 30 Summary: Phase: 3 | Train Loss: 0.4832 | Val Loss: 0.4109
Val Loss did not improved. Patience: 12/12
Early Stopping !


In [8]:
# loss plot
def save_loss_plot(train_losses, val_losses, folder_path):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss', color='blue', marker='o')
    plt.plot(val_losses, label='Validation Loss', color='red', marker='x')
    plt.title('Training and Validation Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)


    plot_path = os.path.join(folder_path, "loss_plot.png")
    plt.savefig(plot_path)
    plt.close()
    print(f"--- Loss plot updated and saved to: {plot_path} ---")

save_loss_plot(all_train_losses, all_val_losses, "/content/drive/MyDrive/ParkModeli")

--- Loss plot updated and saved to: /content/drive/MyDrive/ParkModeli/loss_plot.png ---
